# Step 2: Building our chat framework

## Get previously built model for getting intents

In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

/home/manparvesh/anaconda3/envs/nlp-yoda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We’ll build a simple state-machine to handle responses, using our intents model (from the previous step) as our classifier. [That’s how chatbots work](https://medium.freecodecamp.com/how-chat-bots-work-dfff656a35e2).

> A contextual chatbot framework is a classifier within a state-machine.

- we’ll un-pickle our model and documents as well as reload our intents file
-  Remember our chatbot framework is separate from our model build — you don’t need to rebuild your model unless the intent patterns change
- With several hundred intents and thousands of patterns the model could take several minutes to build

In [2]:
# restore all of our data structures
import pickle
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Util functions

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return(np.array(bag))

In [7]:
p = bow("is your shop open today?", words)
print(p)
print(classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [8]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /home/manparvesh/Desktop/github_repos/chatbot/model.tflearn


Checking that our model gives the same result as in the previous step

In [12]:
def get_predicted_intent(predictions):
    return classes[np.argmax(predictions)]


assert 'opentoday' == get_predicted_intent(model.predict([p]))

## Response processor

In [13]:
# data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25


def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]

    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append((classes[r[0]], r[1]))

    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details:
                            print('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details:
                            print('tag:', i['tag'])

                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [14]:
classify('is your shop open today?')

[('opentoday', 0.75395197)]

In [15]:
response('is your shop open today?')

Our hours are 9am-9pm every day


In [16]:
response('do you take cash?')

We accept most major credit cards


In [17]:
context

{}

In [18]:
response('I\'d like to rent a moped')

Are you looking to rent today or later this week?


In [19]:
context

{'123': 'rentalday'}

In [21]:
response('tomorrow would be great')

Hello, thanks for visiting


haha

In [22]:
context

{'123': ''}

In [23]:
response('I\'d like to rent a moped')

Are you looking to rent today or later this week?


In [24]:
context

{'123': 'rentalday'}

In [25]:
response('today please')

For rentals today please call 1-800-MYMOPED


In [26]:
classify('today')

[('today', 0.7277708)]

In [27]:
# clear context
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [32]:
response('today', show_details=True)
classify('today')

[('today', 0.7277708)]

In [33]:
response('THanks, that was fun')

My pleasure
